In [75]:
import pandas as pd
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [76]:
# ID & Secret file
api_info = pd.read_excel('../logins.xlsx')

In [77]:
def create_spotipy_client(client_id, client_secret, redirect_uri, scope):
    '''
    Creates spotipy client object with authorization access to user's designated Spotipy application. Returns spotipy client object.

    ARGUMENTS:
        client_id: User's app client_id as seen via Spotify dashboard.
        client_secret: User's app client_secret as seen via Spotify dashboard.
        redirect_uri: Redirect URL used for acquiring authorization code.
        scope: Scope of accessibility for user's account.
    '''
    sp = spotipy.Spotify(
        auth_manager=SpotifyOAuth(
            client_id=client_id,
            client_secret=client_secret,
            redirect_uri=redirect_uri,
            scope=scope
        )
    )
    return sp

In [78]:
def get_recent_played_tracks(spotipy_client, limit):
    '''
    Gets 50 most recently played songs from user's Spotify account. Spotipy will automatically grab authorization code, swap for access token, and contains a method that returns 50 most recently played songs from user's account. Returns user's most recently played tracks in JSON format.

    ARGUMENTS:    
        spotipy_client: Spotipy client object.
        limit: The number of songs recently played that the user would like to return.
    '''
    return spotipy_client.current_user_recently_played(limit=limit)
    

In [79]:
def get_access_token():
    '''
    Uses Spotipy customized cache handler to acquire access token directly. Returns access token.

    ARGUMENTS:
        None
    '''
    handler = spotipy.CacheFileHandler()
    return handler.get_cached_token()['access_token']

In [80]:
def additional_info(token, artist_id, album_id):
    '''
    Interacts directly with Spotify Web API using access token associated with spotipy client object to acquire additional information for albums.
    Returns artist followers, artist popularity, album popularity, album total tracks, and album release date.

    ARGUMENTS:
        token: Access token required to access user's Spotify app.
        album_id: Unique Spotify ID associated with album of most recent song played.
        artist_id: Unique Spotify ID associated with artist of most recent song played.
    '''
    # Authentican headers
    auth_headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {token}'
    }

    # Concatenate specific artist and album IDs to get request URL
    get_artist_info_url = f'https://api.spotify.com/v1/artists/{artist_id}'
    get_album_info_url = f'https://api.spotify.com/v1/albums/{album_id}'

    # Returned JSON data on specified artist and album
    artist_info = requests.get(get_artist_info_url, headers=auth_headers).json()
    album_info = requests.get(get_album_info_url, headers=auth_headers).json()
    
    # Additional artist metrics
    artist_followers = artist_info['followers']['total']
    artist_popularity = artist_info['popularity']
    
    # Additional album metrics
    album_popularity = album_info['popularity']
    album_total_tracks = album_info['total_tracks']
    album_release_date = album_info['release_date']

    # Returned values
    return artist_followers, artist_popularity, album_popularity, album_total_tracks, album_release_date

In [81]:
spotify_client_id = api_info['client_id'][0]
spotify_client_secret = api_info['client_secret'][0]
spotify_redirect_url = "http://localhost:7777/callback"

sp = create_spotipy_client(client_id=spotify_client_id, client_secret=spotify_client_secret, redirect_uri=spotify_redirect_url, scope='user-read-recently-played')

In [82]:
recent_tracks = get_recent_played_tracks(sp, 50)
recent_tracks

{'items': [{'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3dz0NnIZhtKKeXZxLOxCam'},
       'href': 'https://api.spotify.com/v1/artists/3dz0NnIZhtKKeXZxLOxCam',
       'id': '3dz0NnIZhtKKeXZxLOxCam',
       'name': 'Porter Robinson',
       'type': 'artist',
       'uri': 'spotify:artist:3dz0NnIZhtKKeXZxLOxCam'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'DM',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',

In [59]:

for i in range(len(recent_tracks['items'])):
    artist_id = recent_tracks['items'][i]['track']['album']['artists'][0]['id']
    album_id = recent_tracks['items'][i]['track']['album']['id']
    artist_followers, artist_popularity, album_popularity, album_total_tracks, album_release_date = additional_info(token=get_access_token(), artist_id=artist_id, album_id=album_id)
    print('Artist ID: ', recent_tracks['items'][i]['track']['album']['artists'][0]['id'], '\nArtist Name: ', recent_tracks['items'][i]['track']['album']['artists'][0]['name'], '\nTrack Name: ', recent_tracks['items'][i]['track']['name'], '\nArtist URL: ', recent_tracks['items'][i]['track']['album']['artists'][0]['external_urls']['spotify'], '\nArtist Followers: ', artist_followers)
    print()

Artist ID:  4FrFdi2hHpulQOxDihRIQD 
Artist Name:  Aoi Teshima 
Track Name:  しずかだなあ 
Artist URL:  https://open.spotify.com/artist/4FrFdi2hHpulQOxDihRIQD 
Artist Followers:  181232

Artist ID:  5na1LmEmK2VzNLje9snJYW 
Artist Name:  The Rose 
Track Name:  Childhood 
Artist URL:  https://open.spotify.com/artist/5na1LmEmK2VzNLje9snJYW 
Artist Followers:  2042303

Artist ID:  5sqUW4XEHsGVQT6z5NGgJf 
Artist Name:  Nao'ymt 
Track Name:  孤島 
Artist URL:  https://open.spotify.com/artist/5sqUW4XEHsGVQT6z5NGgJf 
Artist Followers:  5942

Artist ID:  0LyfQWJT6nXafLPZqxe9Of 
Artist Name:  Various Artists 
Track Name:  Letter 
Artist URL:  https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of 
Artist Followers:  1814896

Artist ID:  0LyfQWJT6nXafLPZqxe9Of 
Artist Name:  Various Artists 
Track Name:  A Million Miles Away - reprise 
Artist URL:  https://open.spotify.com/artist/0LyfQWJT6nXafLPZqxe9Of 
Artist Followers:  1814896

Artist ID:  5na1LmEmK2VzNLje9snJYW 
Artist Name:  The Rose 
Track Name:  so

In [83]:
token = get_access_token()

In [84]:
artist_id = recent_tracks['items'][0]['track']['album']['artists'][0]['id']
album_id = recent_tracks['items'][0]['track']['album']['id']

In [89]:
auth_headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}'
}
get_artist_info_url = f'https://api.spotify.com/v1/artists/{artist_id}'
artist_info = requests.get(get_artist_info_url, headers=auth_headers).json()
artist_followers = artist_info['followers']['total']
# artist_info['followers']['total']
artist_followers

1053601

In [ ]:
artist_name, artist_followers, artist_popularity, album_popularity, album_total_tracks, album_release_date = additional_info(token, artist_id=artist_id, album_id=album_id)

In [20]:
print(artist_followers, artist_popularity, album_popularity, album_total_tracks, album_release_date)

24033 64 48 5 2020-05-10


In [ ]:

# fact table stuff
    # track_id = recent_songs_request['items'][i]['track']['id]
    # artist_id = recent_songs_request[                 'items'][i]['track']['album']['artists'][0]['id']
    # album_id = recent_songs_request['items'][i]['track']['album']['id']
    # track_name = recent_songs_request['items'][i]['track']['name']
    # track_uri = recent_songs_request['items'][i]['track']['uri']
    # track_length_ms = recent_songs_request['items'][i]['track']['duration_ms']
    # track_popularity = recent_songs_request['items'][i]['track']['popularity']
    # played_at = recent_songs_request['items'][0][]'played_at']

# artist stuff
    # artist_id = recent_songs_request['items'][i]['track']['album']['artists'][0]['id']
    # artist_name = recent_songs_request['items'][i]['track']['album']['artists'][0]['name']
    # artist_url = recent_songs_request['items'][i]['track']['artists'][0]['uri']
    # artist_info_requests = requests.get('https://api.spotify.com/v1/artists/5na1LmEmK2VzNLje9snJYW', headers=artist_headers).json()
    # artist_followers = artist_info['followers']['total']
    # artist_popularity = artist_info['popularity']


# album stuff 
    # album_id = recent_songs_request['items'][i]['track']['album']['id']
    # album_name = recent_songs_request['items'][i]['track']['album']['name']
    # album_url = recent_songs_request['items'][i]['track']['album']['uri']
    # album_info = requests.get('https://api.spotify.com/v1/albums/6QX9efMZlYa0HQic1iO3PX', headers=album_headers).json()
    # album_popularity = album_info['popularity']
    # album_total_tracks = recent_songs_request['items'][i]['track']['album']['total_tracks']
    # album_release_date = recent_songs_request['items'][i]['track']['album']['release_date']

# 
# functions for transforming extracted data or use sql stored procedure to change this
    # if album, track, or artist then concatenate
    # 'open.spotify.com/' + '{type_of_data}/' + album/song/track_uri.split(':')[-1]
    # date time needs to be 


# within items []
# id -> song id
# name -> band name 
# json file format hierarchy
#   items -> track -> album / artists 